In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk, ImageOps, ImageDraw
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image as keras_image
import os
import webbrowser
import warnings
import tempfile
import math
import random

# Suppress TensorFlow warnings
warnings.filterwarnings("ignore", category=UserWarning)
tf.get_logger().setLevel('ERROR')

# Color scheme
BG_COLOR = "#e8f5e9"  # Light green background
PRIMARY_COLOR = "#2e7d32"  # Dark green
SECONDARY_COLOR = "#81c784"  # Medium green
ACCENT_COLOR = "#ff8f00"  # Orange for highlights
TEXT_COLOR = "#263238"  # Dark gray for text
LIGHT_TEXT = "#f5f5f5"  # Light text for dark backgrounds

class NatureBackground:
    def __init__(self, root, width, height):
        self.root = root
        self.width = width
        self.height = height
        self.canvas = tk.Canvas(root, width=width, height=height, highlightthickness=0, bg=BG_COLOR)
        self.canvas.place(x=0, y=0)
        
        # Create gradient background
        self.create_gradient()
        
        # Draw nature elements
        self.draw_nature_elements()
        
    def create_gradient(self):
        """Create a soft green gradient background"""
        for i in range(0, self.height, 5):
            ratio = i / self.height
            r = int(232 - (232 - 180) * ratio)
            g = int(245 - (245 - 215) * ratio)
            b = int(233 - (233 - 180) * ratio)
            color = f'#{r:02x}{g:02x}{b:02x}'
            self.canvas.create_rectangle(0, i, self.width, i+5, fill=color, outline="")
    
    def draw_nature_elements(self):
        """Draw decorative nature elements"""
        # Draw leaves in corners
        leaf_color = "#a5d6a7"
        for x, y, size, angle in [
            (50, 50, 80, 45),
            (self.width-100, 50, 80, -45),
            (50, self.height-100, 80, 135),
            (self.width-100, self.height-100, 80, -135)
        ]:
            self.draw_leaf(x, y, size, angle, leaf_color)
        
        # Draw subtle floral pattern
        for _ in range(20):
            x = random.randint(0, self.width)
            y = random.randint(0, self.height)
            size = random.randint(10, 30)
            self.draw_flower(x, y, size)
    
    def draw_leaf(self, x, y, size, angle, color):
        """Draw a leaf shape"""
        points = []
        for i in range(8):
            rad = math.radians(angle + i * 45)
            radius = size * (0.5 + 0.5 * (i % 2))
            points.extend([
                x + radius * math.cos(rad),
                y + radius * math.sin(rad)
            ])
        self.canvas.create_polygon(points, fill=color, outline="", smooth=True)
    
    def draw_flower(self, x, y, size):
        """Draw a simple flower"""
        petal_color = random.choice(["#ffcdd2", "#f8bbd0", "#e1bee7"])
        self.canvas.create_oval(
            x-size//2, y-size//2, x+size//2, y+size//2,
            fill=petal_color, outline=""
        )
        self.canvas.create_oval(
            x-size//4, y-size//4, x+size//4, y+size//4,
            fill="#ffeb3b", outline=""
        )

class PlantDiseaseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🌿 Plant Disease Detector")
        self.root.geometry("1100x750")
        
        # Create beautiful nature background
        self.background = NatureBackground(root, 1100, 750)
        
        # Initialize image references
        self.image_references = []
        
        # Create main container
        self.main_frame = tk.Frame(root, bg='white')
        self.main_frame.place(relx=0.5, rely=0.5, anchor='center', width=1000, height=650)
        
        # Create widgets
        self.create_widgets()
        
        # Load model
        self.model = self.load_model()
        
        # Disease classes
        self.class_names = [
            'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust',
            'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)_Powdery_mildew',
            'Cherry_(including_sour)_healthy', 'Corn_(maize)_Cercospora_leaf_spot Gray_leaf_spot',
            'Corn_(maize)Common_rust', 'Corn_(maize)_Northern_Leaf_Blight', 'Corn_(maize)_healthy',
            'Grape___Black_rot', 'Grape__Esca(Black_Measles)', 'Grape__Leaf_blight(Isariopsis_Leaf_Spot)',
            'Grape___healthy', 'Orange__Haunglongbing(Citrus_greening)', 'Peach___Bacterial_spot',
            'Peach___healthy', 'Pepper,bell__Bacterial_spot', 'Pepper,bell__healthy',
            'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy',
            'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew',
            'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot',
            'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold',
            'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
            'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
            'Tomato___Tomato_mosaic_virus', 'Tomato___healthy'
        ]

    def create_widgets(self):
        """Create all GUI components"""
        # Header
        header_frame = tk.Frame(self.main_frame, bg=PRIMARY_COLOR)
        header_frame.pack(fill=tk.X, pady=(0,10))
        
        tk.Label(
            header_frame, 
            text="🌱 Plant Disease Detector", 
            font=("Helvetica", 20, "bold"), 
            fg=LIGHT_TEXT, 
            bg=PRIMARY_COLOR,
            padx=10,
            pady=10
        ).pack(side=tk.LEFT)
        
        # Image display
        self.image_frame = tk.Frame(
            self.main_frame, 
            bg="white", 
            highlightbackground=SECONDARY_COLOR,
            highlightthickness=2
        )
        self.image_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=5)
        
        self.image_label = tk.Label(
            self.image_frame, 
            text="No image loaded", 
            font=("Arial", 12), 
            bg="white", 
            fg=TEXT_COLOR
        )
        self.image_label.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Control buttons
        control_frame = tk.Frame(self.main_frame, bg='white')
        control_frame.pack(fill=tk.X, padx=20, pady=10)
        
        button_style = {
            'font': ('Arial', 12, 'bold'),
            'borderwidth': 0,
            'relief': tk.FLAT,
            'cursor': 'hand2',
            'padx': 15,
            'pady': 8
        }
        
        tk.Button(
            control_frame, 
            text="📁 Load Image", 
            command=self.load_image,
            bg=PRIMARY_COLOR,
            fg=LIGHT_TEXT,
            activebackground=SECONDARY_COLOR,
            **button_style
        ).pack(side=tk.LEFT, padx=5)
        
        self.detect_btn = tk.Button(
            control_frame, 
            text="🔍 Detect Disease", 
            command=self.detect_disease, 
            state=tk.DISABLED,
            bg=ACCENT_COLOR,
            fg=LIGHT_TEXT,
            **button_style
        )
        self.detect_btn.pack(side=tk.LEFT, padx=5)
        
        self.google_lens_btn = tk.Button(
            control_frame,
            text="🔎 Google Lens",
            command=self.open_google_lens,
            state=tk.DISABLED,
            bg=SECONDARY_COLOR,
            fg=LIGHT_TEXT,
            **button_style
        )
        self.google_lens_btn.pack(side=tk.LEFT, padx=5)
        
        tk.Button(
            control_frame,
            text="❓ Help",
            command=self.show_help,
            bg=SECONDARY_COLOR,
            fg=LIGHT_TEXT,
            **button_style
        ).pack(side=tk.LEFT, padx=5)
        
        # Results display
        result_frame = tk.Frame(
            self.main_frame, 
            bg="white",
            highlightbackground=SECONDARY_COLOR,
            highlightthickness=2
        )
        result_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=5)
        
        scrollbar = tk.Scrollbar(result_frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.result_text = tk.Text(
            result_frame, 
            wrap=tk.WORD, 
            font=("Arial", 12), 
            bg="white",
            yscrollcommand=scrollbar.set,
            padx=10,
            pady=10
        )
        self.result_text.pack(fill=tk.BOTH, expand=True)
        scrollbar.config(command=self.result_text.yview)
        
        # Status bar
        self.status_var = tk.StringVar(value="Ready to analyze plant images")
        tk.Label(
            self.main_frame,
            textvariable=self.status_var,
            font=("Arial", 10),
            fg=LIGHT_TEXT,
            bg=PRIMARY_COLOR,
            anchor=tk.W
        ).pack(fill=tk.X, padx=20, pady=(5,0))

    def load_model(self):
        """Load the disease detection model"""
        try:
            if os.path.exists("final_model.h5"):
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    model = tf.keras.models.load_model("final_model.h5")
                    model.compile(optimizer='adam', 
                                loss='categorical_crossentropy', 
                                metrics=['accuracy'])
                    return model
            messagebox.showerror("Error", "Model file not found!")
            self.root.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load model: {str(e)}")
            self.root.destroy()

    def load_image(self):
        """Load an image file"""
        filepath = filedialog.askopenfilename(
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        if filepath:
            try:
                self.current_image = Image.open(filepath)
                self.display_image()
                self.detect_btn.config(state=tk.NORMAL)
                self.google_lens_btn.config(state=tk.NORMAL)
                self.status_var.set(f"Loaded: {os.path.basename(filepath)}")
                self.result_text.config(state=tk.NORMAL)
                self.result_text.delete(1.0, tk.END)
                self.result_text.config(state=tk.DISABLED)
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load image: {str(e)}")

    def display_image(self):
        """Display the current image"""
        img = self.current_image.copy()
        img.thumbnail((800, 600))
        img_with_border = ImageOps.expand(img, border=2, fill=PRIMARY_COLOR)
        
        # Create and store PhotoImage
        self.tk_image = ImageTk.PhotoImage(img_with_border)
        self.image_references.append(self.tk_image)  # Maintain reference
        
        self.image_label.config(
            image=self.tk_image,
            compound=tk.CENTER,
            bg="white"
        )

    def detect_disease(self):
        """Detect disease in the loaded image"""
        if not hasattr(self, 'current_image'):
            return
            
        self.detect_btn.config(state=tk.DISABLED, text="Analyzing...")
        self.status_var.set("Analyzing plant image...")
        self.root.update()
        
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete(1.0, tk.END)
        
        try:
            # Prepare image for model
            img = self.current_image.resize((256, 256))
            img_array = keras_image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = img_array / 255.0
                
            # Make prediction
            preds = self.model.predict(img_array)[0]
            pred_idx = np.argmax(preds)
            confidence = preds[pred_idx]
            disease = self.class_names[pred_idx]
                
            if confidence > 0.7:  # Confidence threshold
                self.show_known_disease(disease, confidence)
            else:
                self.handle_unknown_disease()
            
        except Exception as e:
            messagebox.showerror("Error", f"Detection failed: {str(e)}")
            self.status_var.set("Error occurred during analysis")
        finally:
            self.detect_btn.config(state=tk.NORMAL, text="🔍 Detect Disease")
            self.result_text.config(state=tk.DISABLED)

    def show_known_disease(self, disease, confidence):
        """Display results for known diseases"""
        display_name = disease.replace('___', ' ').replace('_', ' ')
        display_name = display_name.replace('(including sour)', '').replace('(maize)', '')
        
        self.result_text.config(state=tk.NORMAL)
        self.result_text.insert(tk.END, "✅ Disease Identified\n\n", "bold_green")
        self.result_text.insert(tk.END, f"🌿 Plant: {display_name.split()[0]}\n")
        self.result_text.insert(tk.END, f"🦠 Disease: {' '.join(display_name.split()[1:])}\n")
        self.result_text.insert(tk.END, f"📊 Confidence: {confidence:.1%}\n\n")
        
        treatment = self.get_treatment_info(disease)
        self.result_text.insert(tk.END, "💊 Treatment Recommendations\n\n", "bold_green")
        self.result_text.insert(tk.END, treatment)
        
        self.result_text.tag_configure("bold_green", 
                                    font=("Arial", 12, "bold"), 
                                    foreground=PRIMARY_COLOR)
        self.status_var.set(f"Identified: {display_name}")

    def get_treatment_info(self, disease):
        """Get treatment information for known diseases"""
        treatments = {
            'Apple___Apple_scab': """🍎 Apple Scab Treatment:
• Apply fungicides early in season
• Prune for air circulation
• Remove fallen leaves in autumn""",
            'Tomato___Early_blight': """🍅 Early Blight Treatment:
• Remove affected leaves
• Apply copper-based fungicides
• Rotate crops yearly""",
            # Add more treatments as needed
        }
        return treatments.get(disease, 
                            "No specific treatment information available.\nConsult a plant disease expert.")

    def handle_unknown_disease(self):
        """Handle unknown diseases"""
        self.result_text.config(state=tk.NORMAL)
        self.result_text.insert(tk.END, "⚠️ Unknown Plant Disease\n\n", "bold_orange")
        self.result_text.insert(tk.END, "Could not confidently identify disease.\n\n")
        self.result_text.insert(tk.END, "Opening Google Lens for further analysis...\n")
        
        self.result_text.tag_configure("bold_orange", 
                                     font=("Arial", 12, "bold"), 
                                     foreground=ACCENT_COLOR)
        self.status_var.set("Unknown disease detected - opening Google Lens")
        self.root.update()
        
        # Automatically open Google Lens
        self.open_google_lens()

    def open_google_lens(self):
        """Open Google Lens with the current image"""
        if not hasattr(self, 'current_image'):
            messagebox.showwarning("Warning", "No image loaded to analyze")
            return
            
        try:
            # Save image to temporary file
            with tempfile.NamedTemporaryFile(suffix='.jpg', delete=False) as tmp:
                self.current_image.save(tmp.name, format='JPEG')
                tmp_path = tmp.name
            
            # Open Google Lens with the image
            webbrowser.open(f'https://lens.google.com/uploadbyurl?url=file://{tmp_path}')
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to open Google Lens: {str(e)}")
            self.status_var.set("Failed to open Google Lens")

    def show_help(self):
        """Show help information"""
        help_text = """🌱 Plant Disease Detector Help

1. Click 'Load Image' to select a plant photo
2. Click 'Detect Disease' to analyze
3. View results and treatment recommendations
4. Use 'Google Lens' button for additional analysis

For best results:
• Use clear, well-lit images
• Focus on affected plant parts"""
        messagebox.showinfo("Help", help_text)

if __name__ == "__main__":
    root = tk.Tk()
    style = ttk.Style()
    style.theme_use('clam')
    style.configure('TFrame', background=BG_COLOR)
    style.configure('TLabel', background=BG_COLOR, foreground=TEXT_COLOR)
    style.configure('TButton', font=('Arial', 12), padding=5)
    
    app = PlantDiseaseApp(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
